# Modelling

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Note: To understand today's material you will need to read the explanations carefully. Make sure you understand each part of the notebook before moving on to the next.

Computational modelling is an attempt to reproduce the behaviour of materials using a computer. Once you have a computational model of your material you can use it to understand the origins of its properties or you can use it to predict ~~the future~~ the behaviour of the material. 

Computational modelling is incredibly powerful in both materials discovery and engineering. If we can successfully model a material behaviour, that means we know the "mathematical recipe" for this material, with which we tinker. For example,  we could then use the model to simulate changing the composition to see if the properties could be improved. In engineering, computational materials models are used everyday to design new jet engines, new aeroplanes, crash-worthy cars etc. They are also used to simulate how the materials in a nuclear plant degrade with time, to help predict the safe life limits of a power plant.

Computational models are being used increasingly often in materials science and, as a materials scientist in training, you should become familiar with some key ideas. In this notebook, we explore the simple example of the elastic behaviour of materials. We will use modelling to explore how the energy of a material changes when we apply a force to change its shape. 

We will relate the macroscopic behaviour of a material to an atomic model of the material. Along the way you will learn some useful techniques and hopefully get a feeling for how your Python skills can come together to simulate material behaviour. 

## The modelling "pipeline"

One of the things we want to show you is the idea of moving through various stages of modelling and simulations: *equation* $\rightarrow$  *model* $\rightarrow$  *algorithm* $\rightarrow$ *simulation*. What do we mean by this?

- *Equation*: this is  a mathematical relationship between physical quantities. This is often an empirical relationship, i.e. comes from experiment. 


- *Model*: here we refer to some picture in our head of what is going on in the real system to give rise to the observed behaviour. For example, we might imagine the nuclei of the atoms being forced to move around and the electrons in the material rearranging themselves into their new preferred state, giving rise to a change in the forces acting on the nuclei. 


- *Algorithm*: here we mean taking the elements of our model and turning them into a set of instructions that can be computed. The algorithm is more abstract or more general than a computer program: a computer program can be thought of as a specific implementation of an algorithm.


- *Simulation*: here we mean taking our algorithm (perhaps implemented as computer code) and running through it in various scenarios. We can then observe how the output from the algorithm behaves as we vary the inputs. We'll do this below with our example of Young's Modulus.

## The equation - Young's Modulus

We start off with an equation determined empirically from experimental data. Here we are going to use data from a tensile test on an Argon crystal (yes, [inert gas crystals are a thing](http://www.tandfonline.com/doi/abs/10.1080/00107517008204820). A thin crystal was stretched and the force required to do this was recorded. The data for this experiment are available in the file 'argon_tension.txt'.

Let's load the data into some variables:

In [ ]:
data=np.loadtxt('Data/argon_tension.txt')
stretch=data[:,0]
load=data[:,1]

### <span style="color: red"> Task:</span>   Plot the force against stretch using points as markers
It should look something like this:
<figure>
<img src="Figures/Plot1.jpg" width='200'>
<figcaption></figcaption>
</figure>


### <span style="color: blue"> Solution:</span>

As you can see, the data lie more or less on a straight line, that is, the force needed to stretch the material is proportional to the amount of stretch (defined by $\epsilon = \Delta L/L$., i.e. $\Delta L \propto F$. This is a characteristic of elastic deformation, during which bonds are stretched but the arrangement of the atoms doesn't change.

The slope of this line gives the stiffness of the crystal sample but is not a property of the material since a crystal with a bigger cross-section has a higher stiffness. To turn stiffness into a material property we need to divide by the cross sectional area. By doing this, the force becomes a stress, which is defined as $\sigma = F/A$. Our sample had a cross sectional area of 1 $mm^2$. Let's calculate the stresses from our load values:


In [ ]:
stress=load/1.0e-6

We can then define the property we are interested in: the *elastic modulus* for the material, also called the *Young's Modulus*, $Y$: 

$$
Y = \frac{\sigma}{S} = \frac{F/A}{\Delta L/L}.
$$

The various quantities in this equation are illustrated in the figure below. $L_0$ is the starting length of the crystal (with zero load), $L$ is the stretched length, and $\Delta L$ is the difference between the two. $A$ is the cross sectional area.

<figure>
<img src="Figures/YoungsModulus.jpg" width='200'>
<figcaption></figcaption>
</figure>

We then have a very simple analytical model for the elastic response of a material to an external force:

$$
\sigma = Y{S}.
$$

We can now fit this expression to our data and get the Young's modulus of our material:

In [ ]:
def elastic(S,Y):
    return S*Y

In [ ]:
from scipy.optimize import curve_fit
pars,cov=curve_fit(elastic,stretch,stress,p0=[30e9])

The parameter `p0` of `curve_fit` passes initial guesses for the values of the parameters in the function to be fitted (in this case the value of the Young's modulus in `elastic`). If these guesses aren't specified then all the parameters are assumed to be initially equal to 1. In this case, it then takes the curve fitting algorithm a long time to get all the way to the best-fit value, which is a very large number.

In [ ]:
print('Young\'s modulus is: %2.3f GPa.' % (pars[0]/1e9))

### <span style="color: red"> Task:</span>   Plot the fit as a line along with the experimental data as points

It should look something like this:
<figure>
<img src="Figures/Plot2.jpg" width='200'>
<figcaption></figcaption>
</figure>

### <span style="color: blue"> Solution:</span>

## The model - Balls and springs

One way that we could try to understand the above behaviour might be to develop a model for what is happening in the material at some smaller length scale. In this case, we will start to think about the behaviour of individual atoms in the material and the energy of interaction between them (the bonds). This sort of approach is called *reductionism*: we try to reduce the problem to the simplest interactions at the lowest level. Reductionism can be very useful in understanding the behaviour of physical systems, but we must be careful in how we use it - much of the complex behaviour that we observe in the real world *emerges* from quite simple systems, but not in a way that a purely reductionist approach would ever predict.

In this notebook we will use something much simpler: we will imagine the atoms as balls and the bonds between them as springs. This is more of a *toy model* of the system. This is not a pejorative statement: toy models can often be more suitable for teasing out the essential physics behind real-world behaviour than can full-blown realistic models.

We will model our piece of material as a block of 'atoms', joined together by 'springs', which we take to represent the interatomic bonds. We will assume that the energy of our material depends on the energy of the springs, which in turn depends on how much the springs are stretched or compressed, i.e. on the distance between the balls.

In mathematical terms the energy of a spring (bond) can be written as:

$$E_{\textrm{bond}} = V(r),$$

where $r$ is the length of the spring (bond) and V is the function that describes how the bond or spring energy depends on the bond length. 

### A choice of $V(r)$ - The Lennard-Jones potential


Let's choose a form for our bond energy $V(r)$ and use some of the skills you have developed earlier in the module to explore its properties.

The Lennard-Jones potential is a simple model for the energy of interaction between two atoms. It says that if the atoms are a distance $r$ apart, then their energy of interaction will be:

$$ V(r) = 4 \beta \bigg[ \bigg(\frac{\rho}{r}\bigg)^{12}-\bigg(\frac{\rho}{r}\bigg)^{6}\bigg],$$

where $\beta$ and $\rho$ are parameters. This type of potential, where bond energies are a function only of the bond lengths is called a *pair potential*.

### <span style="color: red"> Task:</span> Plot the form of this function, labelling the axes and using the correct units
Assume that $\beta=0.017\,\mathrm{eV}$ and $\rho=2048^{1/6}\,\textrm{angstrom}$ and plot the Lennard-Jones potential $V$ over a range of $r$ from $3.5\,\textrm{angstrom}$ to $7.0\,\textrm{angstrom}$.

It should look something like this:
<figure>
<img src="Figures/Plot3.jpg" width='200'>
<figcaption></figcaption>
</figure>

### <span style="color: blue"> Solution:</span>

From this plot of the potential we can see that the energy has a minimum when the atoms are at a distance of about 4.0 angstrom. This means the energy increases when they are brought closer together AND when they are pulled apart. This is a simple model for an interatomic potential, it combines two important features of real interatomic interactions: repulsion at close range and attraction at slightly longer ranges. These two features are enough to explain a great deal of the behaviour of real materials. As Richard Feynman put it:

<figure>
<img src="Figures/Feynman.jpg" width='100' align='right'>

</figure>

<blockquote>If, in some cataclysm, all of scientific knowledge were to be destroyed, and only one sentence passed on to the next generations of creatures, what statement would contain the most information in the fewest words? I believe it is the atomic hypothesis (or the atomic fact, or whichever you wish to call it) that all things are made of atoms - little particles that move around in perpetual motion, attracting each other when they are a little distance apart, but repelling upon being squeezed into one another. In that one sentence, you will see, there is an enormous amount of information about the world, if just a little imagination and thinking are applied.
(The Feynman Lectures, Vol 1.)
</blockquote>

Now that you have plotted the potential, you should be able to extract two important quantitative details. The potential has a minimum at a particular separation. This is the *equilibrium separation* $r_0$ for a pair of atoms. The energy of a pair of atoms at the equilibrium separation compared with the energy at infinite separation is the *binding energy* of the pair of atoms.

### <span style="color: red"> Task:</span> From your plot, determine the equilibrium separation and the binding energy

Read the values of these quantities from your plot and make a note of them.

### <span style="color: blue"> Solution:</span>


Now let's consider our model crystal - the block of atoms connected by springs - stretched out to a total length $L$ by a force $F$.

<figure>
<img src="Figures/AtomisticModel.jpg" width='450'>
<figcaption></figcaption>
</figure>

The position of the $i^{\mathrm{th}}$ layer of atoms is $x_i$.

All the atoms in a given layer will contribute the same amount to the energy of the crystal and so the total energy $E$ of this model material will be a sum over all the energies of the individual bonds (of which we have $N$ layers):

$$E = \sum_{i=1}^{N-1} n_{\mathrm{layer}} V\big(x_{i}-x_{i-1}\big)$$

where $n_{\mathrm{layer}}$ is the number of atoms in each layer and $(x_{i}-x_{i-1})$ is the bond length of the bonds to the left of layer $i$. We're going to make an additional assumption that the length of the bonds *within* a layer does not change as we stretch the block and so we are going to ignore the contribution of these bonds to the total energy (elastic behaviour is all about forces and forces are dependent on *changes* in energy). Assuming that we have a simple cubic arrangement of atoms with bonds linking nearest neighbours we can then write

$$n_{\mathrm{layer}} = \frac{A}{r_0^2}.$$

## The algorithm - finding the optimal position of the atoms
Our aim now is to take our model of the argon crystal and calculate how its energy varies as we stretch it. Let's imagine that we clamp a layer of atoms at one end of the crystal in a fixed position and then move the layer at the other end a distance $\Delta L$. What happens to the layers of atoms in between? These layers will move around in such a way as to minimise the energy of the crystal as a whole. This is a general principle governing the way our world works: in the absence of external interference (i.e. if a system is *isolated* from any effects of an *environment*) then it will tend to change in such a way that its total energy is reduced, eventually to a minimum.

So now our task is to find out how we need to change the position of the layers of atoms in order to minimise the energy of the crystal. To do this we will use an approach often referred to as *monte carlo*. This sort of approach is named after a famous casino in Monaco (where, as scientists, we will probably never be able to afford to go, but never mind.....) because it makes use of random numbers. What we will do is repeatedly try moving the layers of atoms around by small amounts in a random direction. If the changes we make result in a reduction of the total energy then we will *accept* them and make the changes permanently, otherwise we will *reject* them and return to the previous positions of the layers. After making many random changes with our rule about acceptance or rejection, we should (we hope) arrive at the optimal configuration and the minimum crystal energy.

The process described above is our *algorithm* and we can illustrate it concisely with a flow diagram:

<figure>
<img src="Figures/Algorithm.jpg" width='450'>
<figcaption></figcaption>
</figure>

## The simulation - Calculating the energy of our model crystal

Now we will use python to implement the above algorithm. First of all we need to define a function to calculate the energy of an individual atomic bond:

In [ ]:
def V(r):
    beta = 0.017
    rho = 2048**(1/6)
    return 4.0*beta*((rho/r)**12-(rho/r)**6)

Now let's try creating a crystal of $N=5$ layers of atoms and calculate its energy. First we will generate an array holding the positions of the layers in the unstretched crystal:

In [ ]:
N=5
r0=4.0
atomPos=np.linspace(0,N-1,N)*r0
print(atomPos)

Our layers are evenly spaced at the equilibrium separation. We can then easily calculate the lengths of all the bonds between the layers:

In [ ]:
springLen=np.diff(atomPos)
print(springLen)

Between the five layers we have four bonds, all of length $r_0$. The sum of all the lengths of the bonds between the layers then gives us the length of the whole crystal:

In [ ]:
L0=sum(springLen)
print(L0)

Now let's calculate the energy of our unstretched crystal. We need to assume a value for the cross-sectional area, so let's go for $1.0\,\textrm{ang}^2$. This allows us to calculate the number of atoms in each layer from $n_{\textrm{layer}}=A/r_0^2$. The energy of the crystal is then the sum of the energies of all the bonds between the layers:

In [ ]:
A = 1.0
nlayer = (A/r0**2)
E0 = nlayer*np.sum(V(springLen))
print(E0)

Now let's stretch our crystal. To do this we first move only the last layer of atoms, stretching only the last spring. Let's move it by `1e-2` angstrom.

In [ ]:
deltaL = 1e-2
atomPos[N-1] = atomPos[N-1] + deltaL
print(atomPos)

And then calculate the new energy:

In [ ]:
springLen = np.diff(atomPos)
E2 = nlayer*np.sum(V(springLen))
print(E2)

We can then calculate the change in the energy as a result of shifting the last layer of atoms:

In [ ]:
print(E2-E0)

Moving this last layer of atoms has increased the energy. Intuitively this makes sense. However, it also seems that stretching only one spring is not the best way of distributing the energy. In other words, this block of atoms is not at equilibrium. Intuitively we know that if all the springs are the same, they should stretch by the same amount - this is equilibrium. So next we are going to use the computer to calculate what this equilibrium energy is and what are the corresponding atom positions (or spring lengths).

To do this we will follow our algorithm and move all intermediate atom positions by small amounts, at random, and then calculate the energy of the crystal. If it decreases then we are moving towards equilibrium, if it increases then we are moving away from equilibrium. So we will reject any movements that increase the energy and accept all those that reduce it.

The code below implements this procedure. Refer back to the flow diagram of our algorithm, above, and see how the algorithm and the code relate to one another - the latter is a specific implementation (in python) of the former. 

In [ ]:
def simulation(iterations,deltaL):
    A = 1.0                                  # Cross-sectional area
    r0 = 4.0                                 # Equilibrium bond length
    nlayer = A/r0**2                         # Number of atoms per layer
    N = 5                                    # Number of layers of atoms
    atomPos0 = np.linspace(0,N-1,N)*r0       # Initial unstretched atom positions
    dispStep=1e-5                            # Amount by which to shift atoms at random
    atomPos0[N-1] = atomPos0[N-1]+deltaL     # Shift last layer of atoms by deltaL
    atomPos1=np.copy(atomPos0)               # Create a copy of atom positions in a second array
    springLen0=np.diff(atomPos0)             # Calculate lengths of bonds
    E0=nlayer*np.sum(V(springLen0))          # Calculate energy of crystal with last layer of atoms displaced
    Eb=[E0]                                  # Store energy of crystal
    for i in range(iterations):                                     # Begin loop through algorithm
        randomDisp=np.sign(np.random.random(N-2)-0.5)*dispStep      # Generate Random shifts for each layer of atoms 
        atomPos1[1:N-1]=atomPos0[1:N-1]+randomDisp                  # Apply shifts to layers
        springLen1=np.diff(atomPos1)                                # Calculate new spring lengths
        E1=nlayer*np.sum(V(springLen1))                             # Calculate new energy
        if E1<E0:                                                   # Is energy reduced?
            atomPos0=np.copy(atomPos1)                                   # If yes, keep new positions.... 
            E0=E1                                                        # ...and copy new energy
        Eb.append(E0)                                               # Add energy to list
    return Eb                                # Return list of energies

Now we will use the code above to find the equilibrium energy of our block of crystal when it is stretched by `1e-2` angstrom:

In [ ]:
i=3000
Eb=simulation(i,1e-2)
plt.plot(range(1,i+2),Eb)
plt.xlabel('Iteration number')
plt.ylabel('$E$ (eV)')
plt.title('Evolution of energy of crystal');
E2 = min(Eb)
print('Change in energy is: %2.8f eV.' % (E2-E0))

### <span style="color: red"> Task:</span> Calculate the change in energy of the block when it is extended by $\Delta L = 0.02\,\textrm{angstrom}$.
(Hint: plot out the evolution of the energy to confirm that the energy has settled to a minimum. If it hasn't, you may need to increase the number of iterations that you use.) Your plot of evolution of energy should look a bit like this (but not exactly - we are using random numbers, remember):
<figure>
<img src="Figures/Plot4.jpg" width='200'>
<figcaption></figcaption>
</figure>

### <span style="color: blue"> Solution:</span>

### Calculating the Young's modulus
We now have a way of finding the relaxed state and so the equilibrium energy of our crystal for a given extension $\Delta L$. By using a python loop, we can quickly plot out the equilibrium energy of the crystal as we vary $\Delta L$:


In [ ]:
ext = np.linspace(-0.01,0.01,21)
i = 5000
E=[]
for deltaL in ext:
    Eb=simulation(i,deltaL)
    E.append(min(Eb))
plt.plot(ext,E,'-o')
plt.xlabel('$\Delta L$ (angstrom)')
plt.ylabel('$E$ (eV)')
plt.title('Evolution of energy of model crystal');

So we've explored the *energy* of our model crystal. But the Young's modulus is related to the *force* exerted by the block when it is deformed. This force, of course, is given by the gradient of the energy with respect to the length of the block:

$$ F = \frac{\mathrm{d}E}{\mathrm{d}L}.$$

Since we have an analytical, differentiable expression for the bond energy, we could derive an analytical expression for this force. However, this wouldn't necessarily be possible for all forms of $V(r)$ and so we will now see how to obtain the force using a numerical approximation to differentiation. We can then apply it to our list of equilibrium energies as a function of $\Delta L$.

Assume that we have a function $f(x)$ that we can evaluate for any value of $x$. Recall that we can define the derivative of a function as

$$\frac{\mathrm{d}f}{\mathrm{d}x} = \lim_{\delta x \to 0} \frac{ f(x+\delta x) - f(x)}{\delta x}.$$

We can calculate the quantity $[f(x+\delta x) - f(x)]/\delta x$ numerically. For small enough $\delta x$, this should be a good approximation to ${\mathrm{d}f}/{\mathrm{d}x}$. You may have noticed that this expression is actually asymmetric about $x$. This doesn't matter when we take the limit $\delta x \to 0$, but for a numerical derivative, we can improve the estimate by calculating:

$$\frac{\mathrm{d}f}{\mathrm{d}x} \approx \frac{ f(x+\delta x) - f(x-\delta x)}{2\delta x}.$$

So to approximate the derivative of the energy we need to calculate

$$ \frac{\mathrm{d}E}{\mathrm{d}L} \approx \frac{ E(L+\delta L) - E(L-\delta L)}{2\delta L}.$$

Let's do that now with python and plot the result.

In [ ]:
F = []
for i in range(1,len(ext)-1):
    F.append((E[i+1]-E[i-1])/(ext[i+1]-ext[i-1]))


The above bit of code takes the list of extensions from above and the list of equilibrium energies and then uses them in the formula for the numerical derivative to produce a list of forces at the different extensions. Below we plot the result.

In [ ]:
plt.plot(ext[1:len(ext)-1],F,'o')
plt.xlabel('$\Delta L$ (angstrom)')
plt.ylabel('F (eV/angstrom)')
plt.title('Restoring force on model crystal');

You will notice that for the very small changes in length that we have explored, the force is almost a linear function of $L$. Our model crystal appears to obey Hooke's law for small extensions:

$$ F= -\alpha\Delta L.$$

Let's use python to fit the constant of proportionality (we know how to do this from previous weeks):

In [ ]:
from scipy.optimize import curve_fit

def model(ext, k):       # Try to fit a linear model
    return k*ext 

popt, pcov = curve_fit(model, ext[1:len(ext)-1], F)
print('The fitted constant of proportionality is %2.5f eV/ang^2' % popt)

plt.plot(ext[1:len(ext)-1],F,'ro')
plt.plot(ext,model(ext,popt),'b-')
plt.xlabel('$\Delta L$ (angstrom)')
plt.ylabel('F (eV/angstrom)')
plt.title('Restoring force on model crystal');

### <span style="color: red"> Task:</span> Estimate the Young's modulus $Y$
Estimate the Young's modulus ($Y=\sigma / \epsilon$) in GPa for our crystal based on the above linear fit for $\alpha$ and print it in the cell below. (*Hint: remember that our units are eV for energy, eV/ang for force and ang for length and that $1\mathrm{eV}\approx 1.60\times10^{-19} \mathrm{ Pa\space m^{3}}$.*)

### <span style="color: blue"> Solution:</span>

Compare this result with the Young's modulus that we fitted to the experimental data at the start of the class. The values are very close. Why? How is this simple toy model of a crystal able to give us an answer so close to the experimentally measured constant for the real crystal? 

The answer is that we have deliberately chosen a function for the bond energy that will give us the right answer. The constants $\beta$ and $\rho$ have been chosen so that our toy model reproduces certain behaviour of the real material - in this case the elastic behaviour under small linear extensions. This is called *empirical fitting* of the potential function and is an important aspect of materials modelling that you will learn more about in later years of the course.

If you want to explore some more ideas in computational modelling then take a look at the optional material in the extension notebook.